In [54]:
import os
import pandas as pd
from tqdm import tqdm

In [44]:
# # generate params for step02 snakemake file
# path = '../fastq'
# ls = [os.path.join(path, i) for i in os.listdir(path)]
# ls.sort()
# ls

In [45]:
# ls_fq = []
# for i in ls:
#     ls_this = os.listdir(i)
#     ls_this = [i for i in ls_this if '_R1' in i]
#     ls_fq.append(os.path.basename(i) + '/' + ls_this[0].split('_R1.fastq.gz')[0])
# ls_fq

convert hicpro to h5
```shell
hicConvertFormat \
    -m dixon_2M_10000.matrix \
    --bedFileHicpro dixon_2M_10000_abs.bed \
    --inputFormat hicpro \
    --outputFormat h5 \
    -o matric.h5 \
    --resolutions 10000
```

In [46]:
path = '../out_dir/hic_results/matrix/'
ls = [os.path.join(path, i) for i in os.listdir(path)]
ls.sort()
ls

['../out_dir/hic_results/matrix/dixon_2M',
 '../out_dir/hic_results/matrix/dixon_2M_2']

In [47]:
ls_matrix = []
ls_bed = []
for i in ls:
    basename = os.path.basename(i) # dixon_2M
    path = os.path.join(i, 'raw') # ../out_dir/hic_results/matrix/dixon_2M/raw
    
    ls_this_dir = os.listdir(path)
    # sort
    ls_this_dir = list(map(int, ls_this_dir))
    ls_this_dir.sort()
    ls_this_dir = list(map(str, ls_this_dir))
    
    ls_this_dir = [os.path.join(path, bin_) for bin_ in ls_this_dir] # ../out_dir/hic_results/matrix/dixon_2M/raw/5000
    ls_this_dir_bed = [os.path.join(path, basename + '_' + os.path.basename(path) + '_abs.bed') for path in ls_this_dir]
    ls_this_dir_matrix = [os.path.join(path, basename + '_' + os.path.basename(path) + '.matrix') for path in ls_this_dir]
    ls_bed.extend(ls_this_dir_bed)
    ls_matrix.extend(ls_this_dir_matrix)

In [48]:
# ls_matrix

In [49]:
# ls_bed

In [50]:
# # for test
# matrix = ls_matrix[0]
# bed = ls_bed[0]
# cmd = """hicConvertFormat \
#     -m {matrix} \
#     --bedFileHicpro {bed} \
#     --inputFormat hicpro \
#     --outputFormat h5 \
#     -o {h5} \
#     --resolutions {resolution}""".format(
#     matrix=matrix,
#     bed=bed,
#     h5=matrix.replace('matrix', 'h5'),
#     resolution=matrix.split('/raw/')[1].split('/')[0]
# )
# cmd

In [51]:
ls_cmd = []
for idx in range(len(ls_matrix)):
    matrix = ls_matrix[idx]
    bed = ls_bed[idx]
    cmd = "hicConvertFormat -m {matrix} --bedFileHicpro {bed} --inputFormat hicpro --outputFormat h5 -o {h5} --resolutions {resolution}".format(
        matrix=matrix,
        bed=bed,
        h5=matrix.replace('.matrix', '.h5'),
        resolution=matrix.split('/raw/')[1].split('/')[0]
    )
    ls_cmd.append(cmd)
g_cmd = (cmd for cmd in ls_cmd)
g_cmd

<generator object <genexpr> at 0x2b8077785580>

In [58]:
# for cmd in tqdm(g_cmd):
#     status = os.system(cmd)
#     if status !=0:
#         print('Error')
#         break
df = pd.DataFrame(ls_cmd)
df['1'] = '\nsleep 1\n'
df.

,0,1
0,hicConvertFormat -m ../out_dir/hic_results/mat...,\nsleep 1\n
1,hicConvertFormat -m ../out_dir/hic_results/mat...,\nsleep 1\n
2,hicConvertFormat -m ../out_dir/hic_results/mat...,\nsleep 1\n
3,hicConvertFormat -m ../out_dir/hic_results/mat...,\nsleep 1\n
4,hicConvertFormat -m ../out_dir/hic_results/mat...,\nsleep 1\n
5,hicConvertFormat -m ../out_dir/hic_results/mat...,\nsleep 1\n
6,hicConvertFormat -m ../out_dir/hic_results/mat...,\nsleep 1\n
7,hicConvertFormat -m ../out_dir/hic_results/mat...,\nsleep 1\n
8,hicConvertFormat -m ../out_dir/hic_results/mat...,\nsleep 1\n
9,hicConvertFormat -m ../out_dir/hic_results/mat...,\nsleep 1\n


In [17]:
# normalize
ls_cmd = []
for idx in range(len(ls_matrix)):
    matrix = ls_matrix[idx]
    bed = ls_bed[idx]
    cmd = "hicNormalize -m {h5} --normalize norm_range -o {h5nm}".format(
        h5=matrix.replace('.matrix', '.h5'),
        h5nm=matrix.replace('.matrix', '.0_1_norm_range.h5'),
    )
    ls_cmd.append(cmd)
ls_cmd
g_cmd = (cmd for cmd in ls_cmd)
g_cmd

<generator object <genexpr> at 0x2b8077837b30>

In [18]:
for cmd in tqdm(g_cmd):
    status = os.system(cmd)
    if status !=0:
        print('Error')
        break

16it [01:55,  7.24s/it]


In [21]:
# KR 校正
# 性能消耗很大考虑集群跑
ls_cmd = []
for idx in range(len(ls_matrix)):
    matrix = ls_matrix[idx]
    bed = ls_bed[idx]
    cmd = "hicCorrectMatrix correct -m {h5nm} --correctionMethod KR -o {h5kr}".format(
        h5nm=matrix.replace('.matrix', '.0_1_norm_range.h5'),
        h5kr=matrix.replace('.matrix', '.0_1_norm_range_KR_corrected.h5'),
    )
    ls_cmd.append(cmd)
ls_cmd
g_cmd = (cmd for cmd in ls_cmd)
g_cmd

<generator object <genexpr> at 0x2b8077847120>

In [22]:
for cmd in tqdm(g_cmd):
    status = os.system(cmd)
    if status !=0:
        print('Error')
        break

0it [00:00, ?it/s]INFO:hicexplorer.hicCorrectMatrix:matrix contains 34 data points. Sparsity 0.000.


normalisation factor is 160.437


1it [00:31, 31.37s/it]INFO:hicexplorer.hicCorrectMatrix:matrix contains 60 data points. Sparsity 0.000.


normalisation factor is 112.683


2it [00:43, 25.66s/it]INFO:hicexplorer.hicCorrectMatrix:matrix contains 144 data points. Sparsity 0.000.


...

normalisation factor is 4.06164


INFO:hicexplorer.hicCorrectMatrix:matrix contains 11014 data points. Sparsity 0.000.
15it [01:54,  4.03s/it]

normalisation factor is 1.88957


INFO:hicexplorer.hicCorrectMatrix:matrix contains 12814 data points. Sparsity 0.001.
16it [01:56,  7.27s/it]

normalisation factor is 1.29095
